## Install necessary modules

In [1]:
!pip install langchain_google_genai
!pip install pyngrok
!pip install nest_asyncio
!pip install fastapi
!pip install uvicorn
!pip install langchain
!pip install flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/

## Initialize your ngrok API Key

In [1]:
!ngrok authtoken your_api_key

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## Importing the Necessary Modules

In [2]:
import os
import datetime
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from pydantic import BaseModel
from fastapi import FastAPI, HTTPException
from pyngrok import ngrok
import uvicorn
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
import asyncio
from google.colab import userdata

## Initializing Langchain Agent

In [3]:
# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Load API key and initialize language model
def load_llm():
    key = userdata.get('GEMINI_API_KEY')
    llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=key)
    return llm

# Define tools
def tools_initialization():

    def get_time(input=None): # Retrieves the current time
        hour = datetime.datetime.now().strftime("%I")
        minute = datetime.datetime.now().strftime("%M")
        am_pm = datetime.datetime.now().strftime("%p")
        return f"It is {hour} {minute} {am_pm}"

    def get_date(input=None): # Retrieves the current date
        month = datetime.datetime.now().strftime("%B")
        day = datetime.datetime.now().strftime("%d")
        return f"Today is {month} {day}"

    def get_day(input=None): # Retrieves day of the week
        day = datetime.datetime.now().strftime("%A")
        return f"It is {day}"

    tools = [
        Tool(name="Time", func=get_time, description="Provides the current time."),
        Tool(name="Date", func=get_date, description="Provides the current date."),
        Tool(name="Day", func=get_day, description="Gives the current day of the week.")
    ]

    return tools

# Initialize agent
def agent_initialization(llm, tools):
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    custom_prompt = PromptTemplate(
        template="""
        You are a personal assistant named Glitch.
        You can help with information retrieval, problem solving, scheduling tasks, creative writing, and providing steps to complete a task.
        If possible try to answer within 2 sentences. If it is not possible then you can exceed this limit.
        Focus on understanding user input and respond succinctly.
        If multiple queries are mentioned, answer them in the order mentioned.
        If you don't know the answer, state that clearly.
        Important: Return the output as a string, not in markdown format.

        Current conversation:
        {chat_history}

        User: {input}
        Assistant:
        """,
        input_variables=["chat_history", "input"]
    )

    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=False,
        memory=memory,
        handle_parsing_errors=True,
        agent_kwargs={"system_message": custom_prompt.format(chat_history="", input="")}
    )
    
    return agent

## Exposing the Server end-point

In [4]:
# Initialize FastAPI app
app = FastAPI()

class UserMessage(BaseModel):
    message: str

@app.post("/chat")
async def chat(user_message: UserMessage):
    try:
        if user_message.message == "":
            return {"response": "Please enter valid input"}
        response = agent.run(user_message.message)
        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Run the FastAPI server
def run_server():
    ngrok_tunnel = ngrok.connect(8000)
    print(f"Public URL: {ngrok_tunnel.public_url}")

    config = uvicorn.Config(app, port=8000, loop="asyncio")
    server = uvicorn.Server(config)

    with ThreadPoolExecutor() as executor:
        executor.submit(server.run)

    while True:
        try:
            asyncio.sleep(1)
        except KeyboardInterrupt:
            break
    async def start_server():
        await server.serve()

## Start the server

In [ ]:
if __name__ == "__main__":
    llm = load_llm()
    tools = tools_initialization()
    agent = agent_initialization(llm, tools)
    run_server()